**Set environment**

In [1]:
source ../config_duke.sh; show_env
source ../config_load_module_bedtools.sh

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



## Intersect sliding windows with fragments and count

**Test loop**

In [2]:
ls -d ${FD_RES}/data/*/target_PER1.bed

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_Dex/target_PER1.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/T

In [3]:
ls -d ${FD_RES}/data/*/chr17.bed

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_Dex/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_DMSO/chr17.bed
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data

In [4]:
ls -d ${FD_RES}/data/{Input?*,TFX?_DMSO,TFX?_Dex}

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_Dex
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX2_DMSO
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX3_Dex
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX3_DMSO
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/TFX4_Dex
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data

In [9]:
ls -d ${FD_RES}/data/{Input?*,TFX?_DMSO,TFX?_Dex} | wc -l

18


## Intersect PER1

In [43]:
sbatch -p ${NODE} \
    --mem 10G \
    -o ${FD_LOG}/sliding_windows_per1.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#TARGET=${CHROM}
CHROM=chr17
TARGET=target_PER1

### fragments
FD_BEDS=($(ls -d ${FD_RES}/data/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FN_BED=${TARGET}.bed

### sliding windows
FD_WIN=${FD_RES}/sliding_window
FN_WIN=sliding_PER1_len_1kb_gap_100bp.bed

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; intersect fragment and sliding windows"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_WIN}/sliding_PER1_len_1kb_gap_100bp/${SAMPLE}
    FN_OUT_FRG=${TARGET}_fragment.bed.gz
    FN_OUT_CNT=${TARGET}_count.bed.gz
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Input  file: " ${FD_WIN}/${FN_WIN}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    cat  ${FD_BED}/${FN_BED} | head -n 3 
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### intersect sliding window and fragments
    bedtools intersect -a ${FD_WIN}/${FN_WIN} -b ${FD_BED}/${FN_BED} -wo |\
        gzip -c > ${FD_OUT}/${FN_OUT_FRG}
    
    ### count the fragments foreach sliding window
    zcat ${FD_OUT}/${FN_OUT_FRG} |\
        cut -f1-3 |\
        sort -k 1,1 -k2,2n |\
        uniq -c |\
        awk '{$(NF+1)=$1;$1=""}1'   |\
        sed -e 's/^[[:space:]]*//'  |\
        sed -e 's/[[:space:]]/\t/g' |\
        gzip -c \
        > ${FD_OUT}/${FN_OUT_CNT}
        
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT_FRG}
    zcat ${FD_OUT}/${FN_OUT_FRG} | head -n 3
    echo
    echo ${FD_OUT}/${FN_OUT_CNT}
    zcat ${FD_OUT}/${FN_OUT_CNT} | head -n 3
    echo
done
echo "Done!"

EOF

Submitted batch job 28618440


## Intersect Chr17 (No restriction)

In [27]:
sbatch -p ${NODE} \
    --array 0-17 \
    --mem 10G \
    -o ${FD_LOG}/sliding_windows_chr17.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#TARGET=${CHROM}
CHROM=chr17
TARGET=chr17

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### sliding windows
WINDOW=sliding_chr17_len_1kb_gap_100bp
FD_WIN=${FD_RES}/sliding_window
FN_WIN=${WINDOW}.bed

### fragments
FD_BEDS=($(ls -d ${FD_RES}/data/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=${TARGET}.bed

### output
SAMPLE=$(basename ${FD_BED})
FD_OUT=${FD_WIN}/${WINDOW}/${SAMPLE}
FN_OUT_FRG=${TARGET}_fragment.bed.gz
FN_OUT_CNT=${TARGET}_count.bed.gz

### print input Message
echo ++++++++++++++++++++++++++++++++++++++++++++++++
echo "SAMPLE:      " ${SAMPLE}
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Input  file: " ${FD_WIN}/${FN_WIN}
echo "Output file: " ${FD_OUT}/${FN_OUT_FRG}
echo "Output file: " ${FD_OUT}/${FN_OUT_CNT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
cat  ${FD_BED}/${FN_BED} | head -n 3 
echo
echo ${FD_WIN}/${FN_WIN}
cat  ${FD_WIN}/${FN_WIN} | head -n 3 

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### intersect sliding window and fragments
bedtools intersect -a ${FD_WIN}/${FN_WIN} -b ${FD_BED}/${FN_BED} -wo |\
    gzip -c > ${FD_OUT}/${FN_OUT_FRG}

### count the fragments foreach sliding window
zcat ${FD_OUT}/${FN_OUT_FRG} |\
    cut -f1-3 |\
    sort -k 1,1 -k2,2n |\
    uniq -c |\
    awk '{$(NF+1)=$1;$1=""}1'   |\
    sed -e 's/^[[:space:]]*//'  |\
    sed -e 's/[[:space:]]/\t/g' |\
    gzip -c \
    > ${FD_OUT}/${FN_OUT_CNT}

### print output Message
echo "Show the first few lines of the output file"
echo ${FD_OUT}/${FN_OUT_FRG}
zcat ${FD_OUT}/${FN_OUT_FRG} | head -n 3
echo
echo ${FD_OUT}/${FN_OUT_CNT}
zcat ${FD_OUT}/${FN_OUT_CNT} | head -n 3
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28646043


In [32]:
cat ${FD_LOG}/sliding_windows_chr17.1.txt

Slurm Array Index:  1
Time Stamp:         07-19-22+04:55:53

++++++++++++++++++++++++++++++++++++++++++++++++
SAMPLE:       Input1_20x
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr17.bed
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input1_20x/chr17_fragment.bed.gz
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp/Input1_20x/chr17_count.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr17.bed
chr17	107410	108464
chr17	159026	160040
chr17	159426	160303

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp.bed
chr17	0	1000
chr17	100	1100
chr17	200	1200
Show the first few lines of the output file
/gpfs/fs1/

## Intersect Chr17 (filter reciprocal 50%)

In [33]:
sbatch -p ${NODE} \
    --array 0-17 \
    --mem 10G \
    -o ${FD_LOG}/sliding_windows_chr17.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
#CHROMS=($(seq 1 22) X Y)
#CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
#TARGET=${CHROM}
CHROM=chr17
TARGET=chr17

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### sliding windows
WINDOW=sliding_chr17_len_1kb_gap_100bp
FD_WIN=${FD_RES}/sliding_window
FN_WIN=${WINDOW}.bed

### fragments
FD_BEDS=($(ls -d ${FD_RES}/data/{Input?*,TFX?_DMSO,TFX?_Dex}/))
FD_BED=${FD_BEDS[${SLURM_ARRAY_TASK_ID}]}
FN_BED=${TARGET}.bed

### output
###     intersect requirement: A _OR_ B: 
###     either 50% of A is covered OR 50% of B is covered
SAMPLE=$(basename ${FD_BED})
FD_OUT=${FD_WIN}/${WINDOW}_f50_OR_F50/${SAMPLE}
FN_OUT_FRG=${TARGET}_fragment.bed.gz
FN_OUT_CNT=${TARGET}_count.bed.gz

### print input Message
echo ++++++++++++++++++++++++++++++++++++++++++++++++
echo "SAMPLE:      " ${SAMPLE}
echo "Input  file: " ${FD_BED}/${FN_BED}
echo "Input  file: " ${FD_WIN}/${FN_WIN}
echo "Output file: " ${FD_OUT}/${FN_OUT_FRG}
echo "Output file: " ${FD_OUT}/${FN_OUT_CNT}
echo
echo "Show the first few lines of the input file"
echo ${FD_BED}/${FN_BED}
cat  ${FD_BED}/${FN_BED} | head -n 3 
echo
echo ${FD_WIN}/${FN_WIN}
cat  ${FD_WIN}/${FN_WIN} | head -n 3 

### init: create output folder if not exist
mkdir -p ${FD_OUT}

### intersect sliding window and fragments
bedtools intersect -a ${FD_WIN}/${FN_WIN} -b ${FD_BED}/${FN_BED} -wo -f 0.5 -F 0.5 -e |\
    gzip -c > ${FD_OUT}/${FN_OUT_FRG}

### count the fragments foreach sliding window
zcat ${FD_OUT}/${FN_OUT_FRG} |\
    cut -f1-3 |\
    sort -k 1,1 -k2,2n |\
    uniq -c |\
    awk '{$(NF+1)=$1;$1=""}1'   |\
    sed -e 's/^[[:space:]]*//'  |\
    sed -e 's/[[:space:]]/\t/g' |\
    gzip -c \
    > ${FD_OUT}/${FN_OUT_CNT}

### print output Message
echo "Show the first few lines of the output file"
echo ${FD_OUT}/${FN_OUT_FRG}
zcat ${FD_OUT}/${FN_OUT_FRG} | head -n 3
echo
echo ${FD_OUT}/${FN_OUT_CNT}
zcat ${FD_OUT}/${FN_OUT_CNT} | head -n 3
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28670058


In [34]:
cat ${FD_LOG}/sliding_windows_chr17.1.txt

Slurm Array Index:  1
Time Stamp:         07-24-22+04:18:37

++++++++++++++++++++++++++++++++++++++++++++++++
SAMPLE:       Input1_20x
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr17.bed
Input  file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp.bed
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp_f50_OR_F50/Input1_20x/chr17_fragment.bed.gz
Output file:  /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp_f50_OR_F50/Input1_20x/chr17_count.bed.gz

Show the first few lines of the input file
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x//chr17.bed
chr17	107410	108464
chr17	159026	160040
chr17	159426	160303

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/sliding_window/sliding_chr17_len_1kb_gap_100bp.bed
chr17	0	1000
chr17	100	1100
chr17	200	1200
